# AlgoBulls Python Developer Coding Assignment (Internshala)

### Design a simple Algorithmic Trading Strategy

###### By NANDAM TEJAS

In [548]:
!pip install pandas
!pip install numpy
!pip install alpha_vantage
!pip install matplotlib
!pip install plotly
!pip install python-dotenv

!pip freeze > requirements.txt

In [550]:
#import all modules
import os
from datetime import datetime
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
import plotly.graph_objects as go

from dotenv import load_dotenv
load_dotenv()

API_KEY = os.environ.get("API_KEY")

### To define a class `ScriptData` which fetches US Stocks using `alpha_vantage` package
##### And the class contains:
- `fetch_intraday_data` `(method arguments: script)`: Which fetches intraday data for given scripts.
- `convert_intraday_data` `(method arguments: script)`: Which converts the fetched data to pandas Dataframe  

In [351]:
# ScriptData
class ScriptData:
    
    def __init__(self): 
        self.api_key = API_KEY
        self.scripts = {}
        
    def __getitem__(self, script):
        return self.scripts.get(script.upper(), None)
    
    def __setitem__(self, script, output):
        self.scripts[script.upper()] = output
        
    def __contains__(self, script):
        return script.upper() in self.scripts
    
    def fetch_intraday_data(self, script):
        script = script.upper()
        
        if script not in self:
            # Create the TimeSeries and get the intraday data in dictionary
            ts = TimeSeries(key=self.api_key)
            data, meta_data = ts.get_intraday(script)

            # Now store the data in scripts
            self.__setitem__(script, data)
    
    def convert_intraday_data(self, script):
        script = script.upper()
        
        # Get the data from the `fetch_intraday_data` method
        data = self.__getitem__(script)
        
        # If the data is not in dataframe, convert it
        if not isinstance(data, pd.DataFrame):
            # Convert data to pandas Dataframe in `df` 
            df = pd.DataFrame.from_dict({i: data[i]
                                        for i in data.keys()
                                        for j in data[i].keys()},
                                       orient="index")
            # reset column names
            df.columns = list(map(
                lambda x: x.split()[-1],
                df.columns
            ))
            df.reset_index(names=['timestamp'], inplace=True)
        
            self.__setitem__(script, df)

In [352]:
script_data = ScriptData()

In [353]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data("GOOGL")
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2022-12-20 20:00:00,89.5000,89.7200,89.5000,89.6000,10553
1,2022-12-20 19:45:00,89.3700,89.4700,89.3200,89.4700,2583
2,2022-12-20 19:30:00,89.5100,89.5100,89.4100,89.4100,3153
3,2022-12-20 19:15:00,89.4800,89.5900,89.4800,89.5600,4137
4,2022-12-20 19:00:00,89.3800,89.5300,89.3800,89.5300,11038
...,...,...,...,...,...,...
95,2022-12-19 12:15:00,89.2300,89.3400,89.1350,89.3000,478607
96,2022-12-19 12:00:00,89.1800,89.3700,89.1750,89.2207,571174
97,2022-12-19 11:45:00,89.5700,89.5700,89.0200,89.1800,1052856
98,2022-12-19 11:30:00,89.8800,89.9600,89.4850,89.5600,643820


In [354]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data("AAPL")
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2022-12-20 20:00:00,132.8800,133.0600,132.8400,133.0000,18373
1,2022-12-20 19:45:00,132.9400,132.9700,132.8600,132.8600,8220
2,2022-12-20 19:30:00,132.9900,132.9900,132.8601,132.9500,17901
3,2022-12-20 19:15:00,133.0200,133.0200,132.9200,132.9300,5799
4,2022-12-20 19:00:00,133.0500,133.1000,132.9000,133.0000,32508
...,...,...,...,...,...,...
95,2022-12-19 12:15:00,132.3550,132.9800,132.3400,132.8400,2042342
96,2022-12-19 12:00:00,132.1650,132.6100,132.1600,132.3550,2166891
97,2022-12-19 11:45:00,132.0200,132.2300,131.7400,132.1600,2279557
98,2022-12-19 11:30:00,131.7331,132.1400,131.7331,132.0250,2068828


In [355]:
'GOOGL' in script_data

True

In [356]:
'AAPL' in script_data

True

In [357]:
'NVDA' in script_data

False

### To define function called `indicator1` which takes `df` and `timestamp` as inputs and returns another dataframe with two columns
- `timestamp`: Same as ‘timestamp’ column in ‘df’
- `indicator`: Moving Average of the ‘close’ column in ‘df’. The number of
    elements to be taken for a moving average is defined by ‘timeperiod’. For
    example, if ‘timeperiod’ is 5, then each row in this column will be an average
    of total 5 previous values (including current value) of the ‘close’ column.


In [398]:
def indicator1(df, timestamp=5):
    """
    - `inputs`: `df` and `timestamp` with default value `5`
    - `returns`: Dataframe of two columns
            - `timestamp`: Same as ‘timestamp’ column in ‘df’
            - `indicator`: Moving Average of the ‘close’ column in ‘df’. The number of
                elements to be taken for a moving average is defined by ‘timeperiod’. For
                example, if ‘timeperiod’ is 5, then each row in this column will be an average
                of total 5 previous values (including current value) of the ‘close’ column.
    """
    df = df.copy(deep=False)
    # create the new column in the given Dataframe
    for i in range(0, df.shape[0]-timestamp+1):
        df.loc[df.index[i+timestamp-1], 'indicator'] = round(
            ((df['close'][i:i+timestamp].astype(np.float64).sum()) / timestamp), 3)

    #return the dataframe with `timestamp` and `indicator` columns
    return df[['timestamp', 'indicator']]

In [399]:
indicator1(script_data['GOOGL'], timestamp=5)

,timestamp,indicator
0,2022-12-20 20:00:00,NaN
1,2022-12-20 19:45:00,NaN
2,2022-12-20 19:30:00,NaN
3,2022-12-20 19:15:00,NaN
4,2022-12-20 19:00:00,89.514
...,...,...
95,2022-12-19 12:15:00,89.125
96,2022-12-19 12:00:00,89.117
97,2022-12-19 11:45:00,89.145
98,2022-12-19 11:30:00,89.255


In [400]:
indicator1(script_data['AAPL'], timestamp=5)

,timestamp,indicator
0,2022-12-20 20:00:00,NaN
1,2022-12-20 19:45:00,NaN
2,2022-12-20 19:30:00,NaN
3,2022-12-20 19:15:00,NaN
4,2022-12-20 19:00:00,132.948
...,...,...
95,2022-12-19 12:15:00,132.823
96,2022-12-19 12:00:00,132.741
97,2022-12-19 11:45:00,132.602
98,2022-12-19 11:30:00,132.406


### To define class `Strategy` which
- Fetch intraday historical day (‘df’) using ScriptData class.
We’ll refer to the ‘close’ column of ‘df’ as close_data.
- Compute indicator data on ‘close’ of ‘df’ using indicator1 function.
We’ll refer to the ‘indicator’ column of this data as indicator_data.
- Generate a pandas DataFrame called ‘signals’ with 2 columns:
    i. ‘timestamp’: Same as ‘timestamp’ column in ‘df’
    ii. ‘signal’: This column can have the following values:
        1. BUY (When: If indicator_data cuts close_data upwards)
        2. SELL (When: If indicator_data cuts close_data downwards)
        3. NO_SIGNAL (When: If indicator_data and close_data don’t cut each other)

In [497]:
class Strategy:
    
    def __init__(self, script):
        self.script = script
        self.script_data = None
    
    def get_script_data(self):
        """
        Get the script data from `ScriptData` and store it in self.script_data
        """
        script_data = ScriptData()
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
        self.script_data = script_data[self.script].copy()
        
    def get_signals(self):
        """
        Method used to generate the pandas Dataframe called `signals` with 2 columns:
            - `timestamp`: Same `timestamp` from `self.script_data`
            - `signal`: This column can have following values:
                1. `BUY`
                2. `SELL`
                3. `NO_SIGNAL`
        And prints `signals` Dataframe with only those rows where the signal is either `BUY` or `SELL` 
        """
        # Close data
        close_data = self.script_data[['timestamp', 'close']]
        
        # Indicator data
        indicator_data = indicator1(self.script_data, timestamp=5)
        
        # Create the new Dataframe of name `signals`
        signals = pd.concat([close_data[['timestamp', 'close']], indicator_data[['indicator']]], axis=1)
        
        # Convert close to float64 type
        signals['close'] = signals['close'].astype(np.float64)
        
        # Create diff column in signals Dataframe
        for i in range(0, signals.shape[0]):
            signals.loc[signals.index[i], 'diff'] = np.round(signals['indicator'][i] - signals['close'][i])
        
        # Create `signal` column in signals Dataframe
        flag = np.NaN
        for i in range(0, signals.shape[0]):
            if flag == signals.iloc[i]['diff']:
                set_flag = "NO_SIG"
            else:
                if signals.iloc[i]['indicator'] > signals.iloc[i]['close']:
                    set_flag = "BUY"
                elif signals.iloc[i]['indicator'] < signals.iloc[i]['close']:
                    set_flag = "SELL"
                else:
                    set_flag = "NO_SIG"
                flag = signals.iloc[i]['diff']
            signals.loc[signals.index[i], 'signal'] = set_flag
        
        # print the `signals` Dataframe with `signal` column with [BUY, SELL]
        return signals[(signals['signal'] == 'BUY') | (signals['signal'] == 'SELL')][['timestamp', 'signal']]
            

In [498]:
strategy = Strategy("NVDA")

In [501]:
strategy.get_script_data()

In [502]:
strategy.get_signals()

,timestamp,signal
4,2022-12-20 19:00:00,SELL
23,2022-12-20 14:15:00,SELL
24,2022-12-20 14:00:00,BUY
26,2022-12-20 13:30:00,BUY
28,2022-12-20 13:00:00,SELL
31,2022-12-20 12:15:00,SELL
32,2022-12-20 12:00:00,SELL
34,2022-12-20 11:30:00,BUY
35,2022-12-20 11:15:00,BUY
36,2022-12-20 11:00:00,SELL


### [OPTIONAL] Plot a candlestick chart of `df` and `indicator`


In [535]:
def show_graph(script):
    df = script_data[script.upper()]
    ind = indicator1(df, timestamp=5)

    fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                    open=df['open'], high=df['high'],
                    low=df['low'], close=df['close'], 
                    name="Historical Data"), 

                    go.Scatter(x=ind['timestamp'], y=ind['indicator'],
                    line = dict(color = "gray"), name="SMA")
                ]
            )

    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.show()

In [539]:
show_graph('GOOGL')

In [540]:
show_graph("AAPL")

In [541]:
show_graph("NVDA")